In [ ]:
# import external modules
import os
import sys
import json
import copy
import joblib
import numpy as np
import matplotlib.pyplot as plt
import importlib

# import PixelNMF
thisdir = os.getcwd()
mlserverdir = os.path.join(thisdir, 'mlserver-model')
sys.path.append(mlserverdir)
from pixelnmf import PixelNMF

# import NMF2D
# this is needed to convert the NMF models to the local NMF2D definition,
# rather than the NMF2D class defined elsewhere in this repository (with unwanted dependencies)
from nmf2d import NMF2D

In [ ]:
# set monitoring element names
# todo: better distinction between names and short tags
# update: skip BPix1 because too much degraded (since 2025F).

#menames = ['BPix1', 'BPix2', 'BPix3', 'BPix4']
menames = ['BPix2', 'BPix3', 'BPix4']

In [ ]:
# set path to nmf model files

modeldir = '../output_20250714_consolidation/models'

# set path
nmf_files = {}
for mename in menames: nmf_files[mename] = os.path.abspath(os.path.join(modeldir, f'nmf_model_{mename.upper()}_C-v1.pkl'))
    
# existence check
missing = []
for mename, f in nmf_files.items():
    if not os.path.exists(f): missing.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
else:
    print(json.dumps(nmf_files, indent=2))

In [ ]:
# load nmf models

# note: these models were stored within a specific context, that apparently needs to be imported here in order to be able to load them...
topdir = os.path.abspath(os.path.join(thisdir, '../../../..'))
sys.path.append(topdir)

nmf_models = {}
for mename in menames:
    model = joblib.load(nmf_files[mename])
    nmf_models[mename] = NMF2D.from_other(model)

In [ ]:
# make local norms

normdata_dir = os.path.join('../../preprocessing/normdata')

# set path
local_norm_files = {}
for mename in menames:
    metag = f'PXLayer_{mename[-1]}'
    local_norm_files[mename] = os.path.join(normdata_dir, f'avgme_Run2025B-v1_{metag}.npy')
print(json.dumps(local_norm_files, indent=2))
    
# load
local_norms = {}
for mename in menames:
    local_norms[mename] = np.load(local_norm_files[mename])

In [ ]:
# make loss masks

# set path
loss_masking_files = {}
for mename in menames:
    metag = f'PXLayer_{mename[-1]}'
    zerofrac_file = os.path.join(normdata_dir, f'zerofrac_Run2025B-v1_{metag}.npy')
    loss_masking_files[mename] = zerofrac_file
print(json.dumps(loss_masking_files, indent=2))
                                 
# load
loss_masks = {}
for mename in menames:
    loss_masks[mename] = np.load(loss_masking_files[mename])
    loss_masks[mename] = (loss_masks[mename] < 0.9).astype(bool)

In [ ]:
# make a PixelNMF instance

pnmf = PixelNMF(nmf_models, local_norms=local_norms, loss_masks=loss_masks)

In [ ]:
print(pnmf.nmfs)

In [ ]:
# store to joblib file

joblib.dump(pnmf, 'mlserver-model/pixelnmf.joblib')